# Urban Informatics
# Module 04: Intro to pandas, Part II

In [ ]:
import numpy as np
import pandas as pd

# consistent randomization
np.random.seed(0)

## 1. Selecting and slicing data

In [ ]:
# Operation                       Syntax           Result
#------------------------------------------------------------
# Select column                   df[col]          Series
# Select row by label             df.loc[label]    Series
# Select row by integer location  df.iloc[loc]     Series
# Slice rows                      df[5:10]         DataFrame
# Select rows by boolean vector   df[mask]         DataFrame

In [ ]:
# create a pandas dataframe from the location data set
df = pd.read_csv('data/gps-coords.csv')
df.head()

### Indexing by row/column label(s)

In [ ]:
# get a column from the dataframe by its label with [] indexing - returns column as a pandas series with the dataframe's index
df['city'].head()

In [ ]:
# get multiple columns by their labels by passing a list of column names within the [] operator - returns a dataframe (subset)
li = ['city', 'country']
df[li].head()

In [ ]:
# get a single value using the [] operator for a column label then a row label
df['city'][0]

### Indexing using .loc

In [ ]:
# use .loc to select by row label - returns the row as a series whose index is the dataframe column labels
df.loc[0]

In [ ]:
# use .loc to select by row label and column label
df.loc[0, 'country']

In [ ]:
# 1:3 is a slice of rows with label 1 to label 3
# ['city', 'date'] is a list of column labels
df.loc[1:3, ['city', 'date']]

In [ ]:
# slice by rows and columns
df.loc[1:3, 'date':'country']

In [ ]:
# you can set values with .loc as well
print(df.loc[0, 'city'])
df.loc[0, 'city'] = 'London'
print(df.loc[0, 'city'])

In [ ]:
df.head()

### Indexing using .iloc

In [ ]:
# use .iloc for integer-position based indexing as [row, column]
# get the value from the row in position 3 and the column in position 2 (zero-indexed)
df.iloc[3, 2]

In [ ]:
# you can use .iloc with slices too
# slice rows from position 112 to 115 and columns from position 2 to 4
# iloc is not inclusive, so for example "from 2 to 4" will return positions 2 and 3 (but not 4)
df.iloc[112:115, 2:4]

In [ ]:
# you can use iloc to sample every nth row from a data set
n = 300
df.iloc[range(0, len(df), n)]

In [ ]:
# it's easier to tell the difference between .loc and .iloc if the index labels aren't the same as their positions
df.index = [label**2 for label in df.index]
df.head()

In [ ]:
# this returns the rows with labels between 4 and 9 (.loc is inclusive)
df.loc[4:9]

In [ ]:
# this returns rows with labels in positions 4 through 8 (not through 9, because .iloc is not inclusive)
df.iloc[4:9]

In [ ]:
# now it's your turn
# select the even-numbered rows (by integer position) and all columns that begin with "c"


## 2. Grouping and summarizing

In [ ]:
groups = df.groupby('city')

In [ ]:
# calculate the average latitude observed per city
groups['lat'].mean().head()

In [ ]:
# calculate the average latitude observed in the 10 cities with the most observations
most_observed = groups.size().sort_values(ascending=False).index
groups['lat'].mean().reindex(most_observed).head(10)

In [ ]:
# now it's your turn
# calculate the single westernmost coordinate in each country


## 3. Select rows by value

We introduced this briefly in the previous module, but now we'll explore this topic in more depth, including using logical operators.

In [ ]:
# load a reduced set of gps data
df = pd.read_csv('data/gps-coords-reduced.csv')
df.tail()

In [ ]:
# create a Series of true/false, indicating if each row in the column is equal to some value
(df['city']=='Munich').head()

In [ ]:
# now, select only those rows in the df that match that condition
df[df['city']=='Munich']

In [ ]:
# pandas logical operators are: | for or, & for and, ~ for not
# these must be grouped by using parentheses
# which observations are in spain but not in barcelona?
not_bcn = df[(df['country']=='Spain') & ~(df['city']=='Barcelona')]
not_bcn

In [ ]:
# get the unique city names
not_bcn['city'].unique().tolist()

In [ ]:
# select rows where either the city is munich, or the country is serbia
df[(df['city']=='Munich') | (df['country']=='Serbia')]

In [ ]:
# how many observations are west of the prime meridian?
len(df[df['lon'] < 0])

In [ ]:
# get all rows that contain a city that starts with the letter G
row_mask = df['city'].str.startswith('G')
df[row_mask]

In [ ]:
# select all rows with certain city names by using .isin([list])
row_mask = df['city'].isin(['Munich', 'Berat', 'Maia', 'Sarajevo'])
df[row_mask]

In [ ]:
# now it's your turn
# select all the rows that are either in Portugal or are east of the prime meridian


## 4. Working with date-time values

In [ ]:
# load the location data set, indexed by the date field
# and, parse the dates so they're no longer strings but now rather Python datetime objects
# this lets us do date and time based operations on the data set
dt = pd.read_csv('data/gps-coords.csv', index_col='date', parse_dates=True)
dt.head()

In [ ]:
len(dt)

In [ ]:
# 1759 rows - but is the timestamp index unique?
dt.index.is_unique

In [ ]:
# drop duplicates where values in all columns match
dt = dt.drop_duplicates(inplace=False)
len(dt)

In [ ]:
# now is it unique?
dt.index.is_unique

In [ ]:
# now drop any rows that duplicate another's index label
dt = dt[~dt.index.duplicated(keep='first')]
len(dt)

In [ ]:
# now is it unique?
dt.index.is_unique

In [ ]:
# the .weekday attribute determines which day of the week a date is
# 0 is monday and 6 is sunday, Tue-Sat are 1-5
# what day of the week is each datetime in our dataframe's index?
dt.index.weekday[:100]

In [ ]:
# use bitwise OR to create a boolean vector of which rows are on a weekend
weekend_mask = (dt.index.weekday==5) | (dt.index.weekday==6)
weekend_mask

In [ ]:
weekends = dt[weekend_mask]
weekdays = dt[~weekend_mask]

In [ ]:
hourly_share = pd.DataFrame()

# calculate what share of the weekday observations each hour has
weekday_hourly = weekdays.groupby(weekdays.index.hour).size()
hourly_share['weekday'] = weekday_hourly / weekday_hourly.sum()

# calculate what share of the weekend observations each hour has
weekend_hourly = weekends.groupby(weekends.index.hour).size()
hourly_share['weekend'] = weekend_hourly / weekend_hourly.sum()

# format the x-axis ticks like 0:00 times and plot the data
hourly_share.index = [s + ':00' for s in hourly_share.index.astype(str)]
hourly_share.head()

In [ ]:
# visualize it - we'll learn all about this in the next module
%matplotlib inline
ax = hourly_share.plot(kind='bar')

In [ ]:
# now it's your turn
# select all the rows from the dt DataFrame that occurred between 06:00 AM and 12:00 PM


## 5. Merge and Concatenate

In [ ]:
df1 = pd.read_csv('data/tracts_pop_age.csv', dtype={'GEOID':str}).set_index('GEOID')
df2 = pd.read_csv('data/tracts_white_income.csv', dtype={'GEOID':str})

In [ ]:
print(df1.shape)
df1.columns

In [ ]:
print(df2.shape)
df2.columns

In [ ]:
df1 = df1.sample(1000)
print(df1.shape)
df1.head()

In [ ]:
df2 = df2.sample(1000)
print(df2.shape)
df2.head()

In [ ]:
# merge the dataframes together
df = pd.merge(left=df1, right=df2, how='inner', left_index=True, right_on='GEOID')
df = df.set_index('GEOID')
print(df.shape)
df.head()

What happens if you change the "how" argument to 'left' or 'right' or 'outer'? How do you explain this behavior?

In [ ]:
# create two subset dataframes
df_la = df[df['place_name']=='Los Angeles, CA']
df_sf = df[df['place_name']=='San Francisco, CA']

In [ ]:
# concatenate the subsets back together
df_ca = pd.concat([df_la, df_sf], axis=0)
df_ca

In [ ]:
# now it's your turn
# create two subsets of the dataframe: one containing tracts in Arizona, and one containing tracts in Florida
# then concatenate them back together


## 6. Map and Apply

These methods are useful for mapping/applying a function across elements, rows, and columns of a pandas DataFrame or Series. But they have some important and often confusing differences.

1. `.map()` applies a function element-wise on a Series
2. `.apply()` works on a row or column basis on a DataFrame (specify the axis!), or element-wise on a Series
3. `.applymap()` works element-wise on an entire DataFrame

Let's see what that means in practice with some examples.

In [ ]:
dt = pd.read_csv('data/gps-coords-reduced.csv')
dt.head()

In [ ]:
# parse date string to a datetime object
dt['date'] = pd.to_datetime(dt['date'])
dt.head()

In [ ]:
# you can iterate through a DataFrame using the .iterrows() method
for label, row in dt.loc[50:60].iterrows():
    if row['date'].month > 5:
        print('summer')
    else:
        print('spring')

#### alternatively, .map() applies a function element-wise on a Series

In [ ]:
# create a function
def get_season(date):
    if date.month > 5:
        return 'summer'
    else:
        return 'spring'
    
# then map it to the series
dt['season'] = dt['date'].map(get_season)

In [ ]:
dt.head()

In [ ]:
# or do the same thing all in one line, using a lambda function as .map()'s argument
# you commonly see this with pandas
dt['season'] = dt['date'].map(lambda date: 'summer' if date.month > 5 else 'spring')

A [lambda function](http://www.python-course.eu/lambda.php) is a simple, one-off, anonymous function. You can't call it again later because it doesn't have a name. It just lets you repeatedly perform some operation across a series of values (in our case, a column in our dataframe) using a minimal amount of code. Also notice that the if-else statement is all on one line: this is called a [ternary operator](http://pythoncentral.io/one-line-if-statement-in-python-ternary-conditional-operator/) or an inline-if.

In [ ]:
# now it's your turn
# write a function to determine if the datetime is AM or PM, then map it to the date column
# next, rewrite it as a lambda function


#### .apply() is like .map(), but it works on a row or column basis on an entire DataFrame (specify the axis)

In [ ]:
# create a new DataFrame with fake year data
df = pd.DataFrame({'start_year':[2001, 2002, 2005, 2005, 2006], 
                   'mid_year':[2002, 2010, 2008, 2006, 2014],
                   'end_year':[2012, 2018, 2018, 2016, 2017]})
df

In [ ]:
# function to calculate the difference between the min and max values in a series
def get_difference(vector):
    difference = vector.max() - vector.min()
    return difference

df.apply(get_difference, axis=0)

In [ ]:
# same thing again, using a lambda function
df.apply(lambda x: x.max() - x.min(), axis=0)

In [ ]:
# find the difference between the min and max values in each row (ie, column-wise) and save to a new column
df['difference'] = df.apply(get_difference, axis=1)
df

#### .applymap() works element-wise on an entire DataFrame
This is like doing a .map() to each column in the DataFrame

In [ ]:
# divide every value in the dataframe by two
df.applymap(lambda x: x / 2)

## 7. Hierarchical indexing

In [ ]:
df = pd.read_csv('data/tracts_white_income.csv', dtype={'GEOID':str})
df.head()

In [ ]:
# replace the place_name column with discrete city and state columns
df['city'] = df['place_name'].map(lambda x: x.split(', ')[0])
df['state'] = df['place_name'].map(lambda x: x.split(', ')[1])
df = df.drop(columns=['place_name'])
df.head()

In [ ]:
# create a 3-level hierarchical index and sort it for fast look-up performance
df = df.set_index(['state', 'city', 'GEOID']).sort_index()
df.head()

In [ ]:
# is the multiindex unique?
df.index.is_unique

In [ ]:
# index the top level with a single value
df.loc['CA'].mean()

In [ ]:
# index the top two levels with a tuple
df.loc[('CA', 'Los Angeles')].mean()

In [ ]:
# index the top level, group by the 2nd level, calculate a summary stat
df.loc['CA'].groupby(level=0).mean()

## In-Class Exercise

  1. Load the tracts_pop_age.csv and the tracts_white_income.csv datasets as DataFrames (do not sample or filter them).
  1. Merge them together on the geoid.
  1. Map a function to create a new dummy variable with value of 1 if median income is greater than \$50,000 and 0 otherwise.
  1. How many tracts have a non-hispanic white majority of the population?
  1. What is the average tract-level median income in Boston?
  1. Create a subset of the dataframe that only contains tracts in states with names that begin with the letter "N".
  1. Create a subset of this subset that only contains tracts with median income below \$30,000.